In [26]:
import pandas as pd
meld_db = pd.read_csv('data/melb_data_fe.csv', sep=',')
data_frame = meld_db.copy()
# data_frame.info()
data_frame['Date'] = pd.to_datetime(data_frame['Date'])
# data_frame.info()
data_frame['quarter']=data_frame['Date'].dt.quarter
# display(data_frame['quarter'].value_counts())
columns_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
# Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
# В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.
uniq_limit = 150
for col in data_frame.columns:
  if col not in columns_to_exclude and data_frame[col].nunique() < uniq_limit:
    data_frame[col] = data_frame[col].astype('category')


# display(data_frame.sort_values(by='Date',ascending=False))
# display(data_frame.sort_values(by=['Distance', 'Price']).loc[::10,['Distance','Price']])
# Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, у которых площадь участка существенно больше площади здания, чтобы понять, как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 
# mask1 = data_frame['AreaRatio']< -0.8
# mask2 = data_frame['Type'] == 'townhouse'
# mask3 = data_frame['SellerG'] == 'McGrath'
# data_frame[mask1 & mask2 & mask3].sort_values(by=['Date','AreaRatio'], ascending=[True,False], ignore_index=True).loc[:,['Date','AreaRatio']]


# Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.
# data_frame.sort_values(by='AreaRatio', ascending=False, ignore_index=True).iloc[1558]

# Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.
# mask1 = data_frame['Type'] == 'townhouse'
# mask2 = data_frame['Rooms'] > 2
# filtered_df = data_frame[mask1 & mask2]
# price = filtered_df.sort_values(by=['Rooms','MeanRoomsSquare'] , ascending=[True, False], ignore_index=True).iloc[18]['Price']
# display(price)
# data_frame.groupby(by='Regionname')['Distance'].min().sort_values(ascending=False)

# авайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания:
# data_frame.groupby(by='MonthSale')['Price'].agg(['count','mean', 'max']).sort_values(by='count',ascending=False)
# data_frame.groupby(by='MonthSale')['Price'].agg('describe')

# После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 
# Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

# data_frame.groupby('Regionname')['SellerG'].agg(['nunique',set]).sort_values(by='nunique')

# Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.
# data_frame.groupby(by='Rooms')['Price'].mean().sort_values()


# Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
# data_frame.groupby(by='Regionname')['Lattitude'].agg('describe').sort_values(by='std')

# Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
# Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
# Не забудьте перевести даты в формат datetime.
# data_frame['Date'] = pd.to_datetime(data_frame['Date'])

# filtered_df = data_frame[(data_frame['Date'] >= '2017-05-01') & (data_frame['Date'] <= '2017-09-01')]
# filtered_df.groupby(by='SellerG')['Price'].sum().sort_values()

# data_frame.pivot_table(values='Price', index='Rooms',columns='Type', fill_value=0).round()

# А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной.
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.
# data_frame.pivot_table(values='Price', index='Regionname',columns='Weekend', aggfunc='count' )
# Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:
# data_frame.pivot_table(values="Landsize",index='Regionname',columns='Type', aggfunc=['median', 'mean'],
#     fill_value=0).round()

# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):
# data_frame.pivot_table(values='Price', index=['Method','Type'],columns=['Regionname'], aggfunc='median').round()

# Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, просто последовательно обратившись по имени столбцов:
# pivot = data_frame.pivot_table(values="Landsize",index='Regionname',columns='Type', aggfunc=['median', 'mean'],
#     fill_value=0).round()
# display(pivot['mean']['unit'])
# Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:
# mask1 = pivot['mean']['house'] < pivot['median']['house']
# filterd_pivot = pivot[mask1]
# display(filterd_pivot)

# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
# В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.
# data_frame.pivot_table(values='BuildingArea',index='Type',columns='Rooms' , aggfunc='median')

# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.
data_frame.pivot_table(values='Price', aggfunc='median', index='SellerG', columns='Type').sort_values(by='unit')

C:\Users\ilyae\AppData\Local\Temp\ipykernel_21748\427507479.py:89: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_frame.pivot_table(values='Price', aggfunc='median', index='SellerG', columns='Type').sort_values(by='unit')


Type,house,townhouse,unit
SellerG,,,
Burnham,922500.0,NaN,305000.0
Sweeney,850000.0,728000.0,381000.0
Stockdale,715500.0,550000.0,410000.0
Village,973500.0,727500.0,420000.0
Love,780000.0,846000.0,440000.0
YPA,596250.0,498500.0,443500.0
LITTLE,970000.0,800000.0,447000.0
Raine,745000.0,608000.0,447500.0
Jas,935000.0,699000.0,455000.0
